In [ ]:
%load_ext autoreload
%autoreload 2

from dataclasses import dataclass
from querychains import with_context, Context, OpenAiChatEngine, start_server, AnthropicEngine, QueryEngine, current_context, Tag
import json

def show(obj):
    print(json.dumps(obj, indent=2))

In [ ]:
# Minimal context
with Context("MyContext") as c:
    pass

show(c.to_dict())

In [ ]:
# Context nesting
with Context("MyContext") as c:
    with Context("ChildContext1"):
        pass
    with Context("ChildContext2"):
        pass

show(c.to_dict())

In [ ]:
# Inputs and outputs
with Context("MyContext", inputs={"a": 10, "b": 20}) as c:
    c.set_result(30)

show(c.to_dict())

In [ ]:
# Context decorator
@with_context
def my_function(a, b):
    return a + b

with Context("root") as c:
    my_function(10, 20)
    
show(c.to_dict())

In [ ]:
# Error handling

@with_context
def my_function(a, b):
    raise Exception("Oops")

try:
    with Context("root") as c:
        my_function(10, 20)
except:
    pass

show(c.to_dict())

In [ ]:
# Dataclasses

@dataclass
class Person:
    name: str
    age: int
    
@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

show(c.to_dict())

In [ ]:
@dataclass
class Person:
    name: str
    age: int

@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

show(c.to_dict())

In [ ]:
# LLM queries

engine = OpenAiChatEngine()

with Context("root") as c:
        response = engine.query("Hi are you?")
        engine.query("Is the following text genereted by LLM?\n\n" + response)
    
show(c.to_dict())

In [ ]:
# async LLM queries
import asyncio 

engine1 = OpenAiChatEngine()
engine2 = AnthropicEngine()

@with_context
async def make_queries(engine: QueryEngine):
    response = await engine.aquery("Hi are you?")
    return await engine.aquery("Is this nice response?\n\n" + response)  

with Context("root") as c:
    q1 = make_queries(engine1)
    q2 = make_queries(engine2)

    await q1
    await q2

show(c.to_dict())

In [ ]:
# Tags

with Context("root", tags=["tag1", "tag2"]) as c:
    c.add_tag("exp1")  # Add to a context dynammicaly
    c.add_tag(Tag("success!", color="lightgreen"))  # Add colored tag

show(c.to_dict())

In [ ]:
# get current context

with Context("root") as c:
    with Context("child"):
        current_context().add_tag("tag1")

show(c.to_dict())

In [ ]:
# Events (instant kontext with immediate result)

with Context("root") as c:
    c.add_event("Message to Alice", kind="message", data="Hi, Alice!")

show(c.to_dict())